In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from termcolor import colored

In [2]:
from gensim import models
model = models.Word2Vec.load('../model/fbcomment.w2v')

Using TensorFlow backend.


In [3]:
df = pd.read_csv("fb_comment_token_annotated.tsv",encoding='utf-8',sep='\t')
df['token_arr'] = df['token'].apply(lambda x:eval(x))

In [4]:
scored = df[df['judgeScore']!=-32]
scored['wordNum']= scored['token_arr'].apply(lambda x: len(x))


/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
scored[scored['wordNum']<75]['message'].shape

(6244,)

In [6]:
#Let's use 15 as max_seq_len
max_seq_len = 75
k = 800
lessThanMaxSeqLen = scored['wordNum']<=max_seq_len
scoreddf = scored[lessThanMaxSeqLen]
X = scoreddf['token_arr'].values
y = (scored['judgeScore'][lessThanMaxSeqLen]+2)/4

In [7]:
X_train,X_test , y_train,y_test = train_test_split(X,y.values,test_size=0.2,random_state=42)
X_test,X_valid , y_test,y_valid = train_test_split(X_test,y_test,test_size=0.5,random_state=42)
print("Train set Row: %d"%(X_train.shape[0]))
print("Test set Row: %d"%(X_test.shape[0]))
print("Validate set Row: %d"%(X_valid.shape[0]))

Train set Row: 5004
Test set Row: 626
Validate set Row: 626


In [8]:
y.value_counts()

0.250    2758
0.750    1466
0.500    1181
0.000     335
1.000     259
0.125     171
0.875      86
Name: judgeScore, dtype: int64

In [9]:
mapping = [-1,0,1]
def generate_batch(i,batch_size=32,verbose=False,X=X_train,y=y_train):
    comments = X[i:i+batch_size] 
    avaiRow = comments.shape[0]
    batch_X = np.zeros([batch_size,max_seq_len,k+2])
    batch_Y = y[i:i+batch_size]
    for i,comment in enumerate(comments):
        for j in range(max_seq_len):
            if(max_seq_len-j>len(comment)): # Should not be word yet
                if(verbose):print("PAD",end=' ')
                batch_X[i][j][-1] = 1
            else: 
                if(verbose):print(comment[j-(max_seq_len-len(comment))],end=' ')
                try:
                    batch_X[i][j] = np.concatenate((model[comment[j-(max_seq_len-len(comment))]],np.zeros([2])))
                except Exception:
                    if(verbose):print("UNK",end=' ')
                    batch_X[i][j] = np.concatenate((np.zeros([k+1]),[1]))
        if(verbose):print(batch_Y[i])
    if(verbose):print(i)
    batch_Y = np.array([np.array([i]) for i in batch_Y])
    return (batch_X[:avaiRow],batch_Y[:avaiRow])

import random
(batch_X,batch_Y) = generate_batch(random.randint(0,X_train.shape[0]),verbose=True)

PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD เว่อ ! ! ! !   โทร ยัง จะ ไม่ มี 0.25
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD ราย เดือน เน็ต หมด   มี ซื้อ เพิ่ม   ซัก   3   G   ราคา ไม่ ถึง   100   มี ไหม ครับ   ถ้า ไม่ มี จะ ย้าย ค่าย ละ 0.25
PAD PAD จัดการ ให้ ด่วน เลย คับ   ติดตั้ง เน็ต บ้าน ทรู แล่ว   ใน โปรมี โทรศัพบ้านฟรี  UNK ซิมเน็ต มือถือ ฟรี   ค่า ติดตั้ง อะไร ก็ ฟรี หมด   แต่ สิ่ง ที่ ได้ มา คือ   ตัว ปล่อย สัญญาณ ไวไฟ   และ ก็ กล่อง ทีวี   ติดตั้ง ฟรี   แค่ นี้ หรอ คับ   ไหน ล่ะ คับ   โทรศัพบ้าน UNK   ไหน ล่ะ คับ ซิมเน็ต   ไม่ เห็น เลย คับ   รบกวน ตรวจเชค UNK ด่วน เลย น่ะ คับ 0.25
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD 

In [10]:
#Prepare test and validate set
(test_X,test_Y) = generate_batch(0,10000,X=X_test,y=y_test)
(valid_X,valid_Y) = generate_batch(0,10000,X=X_valid,y=y_valid)
print(test_X.shape)
print(test_Y.shape)

(626, 75, 802)
(626, 1)


## Create a model

In [11]:
import tensorflow as tf
#Define two variables which will hols the input data and the target data.
data = tf.placeholder(tf.float32, [None, max_seq_len,k+2])
target = tf.placeholder(tf.float32, [None, 1])

In [12]:
#Create RNN cell
from tensorflow.python.ops import rnn, rnn_cell
num_hidden = 200
num_layers = 3
cells = []
for i in range(num_layers):
    cell = rnn_cell.BasicLSTMCell(num_hidden,state_is_tuple=True)
    cell2 = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=0.5)
    cells.append(cell2)
cell = rnn_cell.MultiRNNCell(cells , state_is_tuple=True)

In [13]:
val, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)
val = tf.transpose(val,[1,0,2])
last = tf.gather(val,int(val.get_shape()[0])-1)

In [14]:
weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))
prediction = tf.nn.sigmoid(tf.matmul(last,weight)+bias)

In [42]:
allpredictions = tf.map_fn(lambda x: tf.matmul(x,weight)+bias, val, dtype=tf.float32)
allpredictions = tf.transpose(allpredictions,[1,0,2])

In [17]:
loss = tf.reduce_mean(tf.squared_difference(prediction, target))

In [18]:
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(loss)

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [19]:
init_op = tf.global_variables_initializer()

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
sess = tf.Session(config=config)

sess.run(init_op)

In [20]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3, allow_growth=True)
config = tf.ConfigProto(gpu_options=gpu_options,allow_soft_placement=True)
sess = tf.Session(config=config)
print(sess.run(init_op))

None


In [21]:
def toSent(n):
    if(n<0.45):return 0
    elif(n<0.65):return 0.5
    else:return 1

In [22]:
def cal_acc(pred,real):
    assert(pred.shape[0]==real.shape[0])
    pred_label = pred.copy()
    pred_label[(pred_label<0.45)] = 0
    pred_label[(pred_label>0.45)&(pred<0.65)] = 0.5
    pred_label[(pred_label>.65)&(pred<=1)] = 1

    real_label = real.copy()
    real_label[(real_label<0.45)] = 0
    real_label[(real_label>0.45)&(real_label<0.65)] = 0.5
    real_label[(real_label>0.65)&(real_label<=1)] = 1
    return ((pred_label==real_label).sum())/pred_label.shape[0]


In [ ]:
batch_size = 8000
no_of_batches = int(len(X_train)/batch_size)
epoch = 30
for i in range(epoch):
    ptr = 0
    avgAcc = 0
    for j in range(no_of_batches):
        #inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        (batch_X,batch_Y) = generate_batch(ptr)
        ptr+=batch_size
        feed_dict = {data: batch_X, target: batch_Y}
        sess.run(minimize,feed_dict)
        predict = sess.run(prediction,feed_dict)
        avgAcc+= cal_acc(predict,batch_Y)/no_of_batches
        #accuracy = sess.run(acc,{data:test_X,target:test_Y})
        #avgAcc += accuracy/no_of_batches
        print(str(j)+".",end='')
        
    print("\nEpoch - ",str(i))
    print("Avg Accuracy (training set): %.2f"%avgAcc)
    #print("Avg acc: {}".format(round(avgAcc,2)))
    selInt = random.randint(0,test_X.shape[0])
    test_pred = sess.run(prediction,{data:test_X[selInt:selInt+10],target:test_Y[selInt:selInt+10]})
    all_test_pred = sess.run(allpredictions,{data:test_X[selInt:selInt+10],target:test_Y[selInt:selInt+10]})
    print(all_test_pred.shape)
    err = sess.run(loss,{data:test_X[selInt:selInt+10],target:test_Y[selInt:selInt+10]})
    #print("Loss: %f"%err)
    
    validation_pred = sess.run(prediction,{data: test_X, target: test_Y})
    acc = cal_acc(validation_pred,test_Y)
    print("Accuracy (test set): %.2f"%acc)
    print("====================================")
    print("Show test result")

    for i,pred in enumerate(test_pred):
        print(all_test_pred[i][-len(X_test[selInt+i]):])
        if(toSent(pred[0])==toSent(y_test[selInt+i])): print("✓")
        else: 
            print("".join(X_test[selInt+i]),end=' ')
            print(pred[0],end=" Ans: ")
            print(y_test[selInt+i],end=' ')
            print("X")
    print("Acc: %.2f"%cal_acc(test_pred,test_Y[selInt:selInt+10]))
validation_pred = sess.run(prediction,{data: valid_X, target: valid_Y})
incorrect = sess.run(loss,{data: valid_X, target: valid_Y})
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))


In [ ]:
saver = tf.train.Saver()
saver.save(sess, 'lstm_sentiment_model_overfit_75seq',global_step=1000)

In [ ]:
sess.close()
#https://stackoverflow.com/questions/33759623/tensorflow-how-to-save-restore-a-model

In [39]:
print( colored('hello', 'red','on_cyan'), colored('world', 'green'))

hello world
